In [1]:
# pip install pandas numpy lets-plot numerize tqdm sqlalchemy jupysql

In [2]:
import kaggle
import csv
import os
import json
import numpy as np
import pandas as pd
import sqlite3

from tqdm.notebook import tqdm, trange
tqdm.pandas()
from numerize import numerize as nz
from lets_plot import *
from lets_plot.mapping import *
LetsPlot.setup_html()
from dotenv import dotenv_values

# Part II: Build Database

## Reading `csv` file and do some data cleaning

Before building database, we should first read our downloaded csv file to obtain our raw data.

In [3]:
df_baltimore = pd.read_csv('../data/raw/BPD_Part_1_Victim_Based_Crime_Data.csv')
df_vancouver = pd.read_csv('../data/raw/crime.csv')

After reading the raw data, the next step is to perform some data cleaning to make the two datasets comparable including:

- Delete unnecessary columns in dataframe
- Add column 'City' to distinguish two cities
- Adjust the 'CrimeTime' column to make data store in the date format
- Reclassify the crime types
- Drop invalid and NA data 

In [4]:
df_baltimore=df_baltimore.drop(columns=['CrimeCode','Inside/Outside','Weapon','Post','District','Location 1','Premise','Total Incidents'])
df_baltimore['CrimeDate'] = pd.to_datetime(df_baltimore['CrimeDate'])
df_baltimore['City']='Baltimore'
df_baltimore

,CrimeDate,CrimeTime,Location,Description,Neighborhood,Longitude,Latitude,City
0,2017-09-02,23:30:00,4200 AUDREY AVE,ROBBERY - RESIDENCE,Brooklyn,-76.60541,39.22951,Baltimore
1,2017-09-02,23:00:00,800 NEWINGTON AVE,AUTO THEFT,Reservoir Hill,-76.63217,39.31360,Baltimore
2,2017-09-02,22:53:00,600 RADNOR AV,SHOOTING,Winston-Govans,-76.60697,39.34768,Baltimore
3,2017-09-02,22:50:00,1800 RAMSAY ST,AGG. ASSAULT,Carrollton Ridge,-76.64526,39.28315,Baltimore
4,2017-09-02,22:31:00,100 LIGHT ST,COMMON ASSAULT,Downtown West,-76.61365,39.28756,Baltimore
...,...,...,...,...,...,...,...,...
276524,2012-01-01,00:00:00,1400 JOH AVE,LARCENY,Violetville,-76.67195,39.26132,Baltimore
276525,2012-01-01,00:00:00,5500 SINCLAIR LN,LARCENY,Frankford,-76.53829,39.32493,Baltimore
276526,2012-01-01,00:00:00,400 N PATTERSON PK AV,LARCENY,CARE,-76.58497,39.29573,Baltimore
276527,2012-01-01,00:00:00,5800 LILLYAN AV,BURGLARY,Glenham-Belhar,-76.54578,39.34701,Baltimore


In [5]:
df_vancouver['CrimeDate'] = pd.to_datetime(df_vancouver['DAY'].astype(str) + '/' + df_vancouver['MONTH'].astype(str) + '/' + df_vancouver['YEAR'].astype(str), format='%d/%m/%Y')
df_vancouver['CrimeDate'] = df_vancouver['CrimeDate'].dt.strftime('%Y-%m-%d')
df_vancouver = df_vancouver.dropna(subset=['HOUR', 'MINUTE'])
df_vancouver['HOUR'] = df_vancouver['HOUR'].astype(int).astype(str).str.zfill(2)
df_vancouver['MINUTE'] = df_vancouver['MINUTE'].astype(int).astype(str).str.zfill(2)
df_vancouver['CrimeTime'] = pd.to_datetime(df_vancouver['HOUR'] + ':' + df_vancouver['MINUTE'], format='%H:%M').dt.strftime('%H:%M:%S')
df_vancouver=df_vancouver.drop(columns=['YEAR','MONTH','DAY','HOUR','MINUTE','X','Y'])
df_vancouver['City']='Vancouver'
df_vancouver


,TYPE,HUNDRED_BLOCK,NEIGHBOURHOOD,Latitude,Longitude,CrimeDate,CrimeTime,City
0,Other Theft,9XX TERMINAL AVE,Strathcona,49.269802,-123.083763,2003-05-12,16:15:00,Vancouver
1,Other Theft,9XX TERMINAL AVE,Strathcona,49.269802,-123.083763,2003-05-07,15:20:00,Vancouver
2,Other Theft,9XX TERMINAL AVE,Strathcona,49.269802,-123.083763,2003-04-23,16:40:00,Vancouver
3,Other Theft,9XX TERMINAL AVE,Strathcona,49.269802,-123.083763,2003-04-20,11:15:00,Vancouver
4,Other Theft,9XX TERMINAL AVE,Strathcona,49.269802,-123.083763,2003-04-12,17:45:00,Vancouver
...,...,...,...,...,...,...,...,...
530646,Mischief,14XX E HASTINGS ST,Grandview-Woodland,49.281276,-123.074746,2017-01-18,14:44:00,Vancouver
530647,Break and Enter Residential/Other,31XX ADANAC ST,Hastings-Sunrise,49.277420,-123.037595,2017-03-03,09:16:00,Vancouver
530648,Mischief,14XX E 7TH AVE,Grandview-Woodland,49.264163,-123.075129,2017-05-29,22:30:00,Vancouver
530650,Theft from Vehicle,8XX HAMILTON ST,Central Business District,49.278168,-123.117031,2017-06-05,17:00:00,Vancouver


In [6]:
print(df_baltimore.columns.tolist()) 
print(df_vancouver.columns.tolist())

['CrimeDate', 'CrimeTime', 'Location', 'Description', 'Neighborhood', 'Longitude', 'Latitude', 'City']
['TYPE', 'HUNDRED_BLOCK', 'NEIGHBOURHOOD', 'Latitude', 'Longitude', 'CrimeDate', 'CrimeTime', 'City']


In [7]:
df_vancouver.rename(columns={'TYPE': 'Description', 
                    'HUNDRED_BLOCK': 'Location', 
                    'NEIGHBOURHOOD': 'Neighborhood'}, inplace=True)


In [8]:
value_counts = df_baltimore['Description'].value_counts()

print(value_counts)

Description
LARCENY                 60528
COMMON ASSAULT          45518
BURGLARY                42538
LARCENY FROM AUTO       36295
AGG. ASSAULT            27513
AUTO THEFT              26838
ROBBERY - STREET        17691
ROBBERY - COMMERCIAL     4141
ASSAULT BY THREAT        3503
SHOOTING                 2910
ROBBERY - RESIDENCE      2866
RAPE                     1637
HOMICIDE                 1559
ROBBERY - CARJACKING     1528
ARSON                    1464
Name: count, dtype: int64


In [9]:
value_counts = df_vancouver['Description'].value_counts()
print(value_counts)

Description
Theft from Vehicle                                        172700
Mischief                                                   70413
Break and Enter Residential/Other                          60862
Other Theft                                                52167
Theft of Vehicle                                           38418
Break and Enter Commercial                                 33845
Theft of Bicycle                                           25730
Vehicle Collision or Pedestrian Struck (with Injury)       21901
Vehicle Collision or Pedestrian Struck (with Fatality)       254
Name: count, dtype: int64


In [10]:
df_merged = pd.concat([df_baltimore, df_vancouver], ignore_index=True)

In [11]:
value_counts = df_merged['Description'].value_counts()

print(value_counts)

Description
Theft from Vehicle                                        172700
Mischief                                                   70413
Break and Enter Residential/Other                          60862
LARCENY                                                    60528
Other Theft                                                52167
COMMON ASSAULT                                             45518
BURGLARY                                                   42538
Theft of Vehicle                                           38418
LARCENY FROM AUTO                                          36295
Break and Enter Commercial                                 33845
AGG. ASSAULT                                               27513
AUTO THEFT                                                 26838
Theft of Bicycle                                           25730
Vehicle Collision or Pedestrian Struck (with Injury)       21901
ROBBERY - STREET                                           17691
ROBBERY - COM

In [12]:
df_merged['Description'] = df_merged['Description'].replace({'Theft from Vehicle': 'Theft', 
                                                             'Mischief': 'Other',
                                                             'Break and Enter Residential/Other': 'Burglary',
                                                             'LARCENY': 'Theft',
                                                             'Other Theft':'Theft',
                                                             'COMMON ASSAULT': 'Assault',
                                                             'BURGLARY': 'Burglary',
                                                             'Theft of Vehicle': 'Theft',
                                                             'LARCENY FROM AUTO': 'Theft',
                                                             'Break and Enter Commercial': 'Burglary',
                                                             'AGG. ASSAULT':'Assault',
                                                             'AUTO THEFT': 'Theft',
                                                             'Theft of Bicycle': 'Theft',
                                                             'Vehicle Collision or Pedestrian Struck (with Injury)': 'Accident',
                                                             'ROBBERY - STREET':'Robbery',
                                                             'ROBBERY - COMMERCIAL':'Robbery',
                                                             'ASSAULT BY THREAT':'Assault',
                                                             'SHOOTING':'Homicide',
                                                             'ROBBERY - RESIDENCE':'Burglary',
                                                             'RAPE':'Sexual offense',
                                                             'HOMICIDE':'Homicide',
                                                             'ROBBERY - CARJACKING':'Robbery',
                                                             'ARSON':'Other',
                                                             'Vehicle Collision or Pedestrian Struck (with Fatality)': 'Accident'
                                                             })

In [13]:
invalid_rows = df_merged['Longitude'].apply(lambda x: -123.9 < x < -123.8 or -122.9 < x < -122.8 or x < -124.1)
df = df_merged[~invalid_rows]
df['id'] = range(1, len(df) + 1)

C:\Users\celty\AppData\Local\Temp\ipykernel_7008\76879916.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['id'] = range(1, len(df) + 1)


Here is our final dataframe:

In [14]:
df

,CrimeDate,CrimeTime,Location,Description,Neighborhood,Longitude,Latitude,City,id
0,2017-09-02 00:00:00,23:30:00,4200 AUDREY AVE,Burglary,Brooklyn,-76.605410,39.229510,Baltimore,1
1,2017-09-02 00:00:00,23:00:00,800 NEWINGTON AVE,Theft,Reservoir Hill,-76.632170,39.313600,Baltimore,2
2,2017-09-02 00:00:00,22:53:00,600 RADNOR AV,Homicide,Winston-Govans,-76.606970,39.347680,Baltimore,3
3,2017-09-02 00:00:00,22:50:00,1800 RAMSAY ST,Assault,Carrollton Ridge,-76.645260,39.283150,Baltimore,4
4,2017-09-02 00:00:00,22:31:00,100 LIGHT ST,Assault,Downtown West,-76.613650,39.287560,Baltimore,5
...,...,...,...,...,...,...,...,...,...
752814,2017-01-18,14:44:00,14XX E HASTINGS ST,Other,Grandview-Woodland,-123.074746,49.281276,Vancouver,752810
752815,2017-03-03,09:16:00,31XX ADANAC ST,Burglary,Hastings-Sunrise,-123.037595,49.277420,Vancouver,752811
752816,2017-05-29,22:30:00,14XX E 7TH AVE,Other,Grandview-Woodland,-123.075129,49.264163,Vancouver,752812
752817,2017-06-05,17:00:00,8XX HAMILTON ST,Theft,Central Business District,-123.117031,49.278168,Vancouver,752813


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 752814 entries, 0 to 752818
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   CrimeDate     752814 non-null  object 
 1   CrimeTime     752814 non-null  object 
 2   Location      750594 non-null  object 
 3   Description   752814 non-null  object 
 4   Neighborhood  747817 non-null  object 
 5   Longitude     750610 non-null  float64
 6   Latitude      750610 non-null  float64
 7   City          752814 non-null  object 
 8   id            752814 non-null  int64  
dtypes: float64(2), int64(1), object(6)
memory usage: 57.4+ MB


## Build the database

After getting the clean data, what we need to do is to build a database and create one table named `crime_listing`, and store all our data into that table.

In [16]:
DATA_FOLDER = os.path.join('../data/clean')

In [17]:
# Create a SQLite database in the ../data/clean/crime.db
conn = sqlite3.connect(os.path.join(DATA_FOLDER, './crime_db.db'))

In [19]:
# Delete the product_listing table
conn.execute('''
DROP TABLE crime_listing;
''')

In [20]:
df['CrimeTime'] = df['CrimeTime'].astype(str)
df['CrimeDate'] = df['CrimeDate'].astype(str)

C:\Users\celty\AppData\Local\Temp\ipykernel_7008\1602267448.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CrimeTime'] = df['CrimeTime'].astype(str)
C:\Users\celty\AppData\Local\Temp\ipykernel_7008\1602267448.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CrimeDate'] = df['CrimeDate'].astype(str)


In [21]:
# Create our table based on the crime data we have

conn.execute('''
CREATE TABLE IF NOT EXISTS crime_listing (
    CrimeDate TEXT,
    CrimeTime TEXT,
    Location TEXT,
    Description TEXT,
    Neighborhood TEXT,
    Longitude REAL,
    Latitude REAL,
    City TEXT,
    id INTEGER TEXT PRIMARY KEY
)
''')


In [22]:
df.to_sql(name='crime_listing', con=conn, if_exists='append', index=False)

752814

In [23]:
!pip freeze > requirement.txt